In [5]:
import csv

In [2]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

Genome = {}
def Genomictabulator(fasta):
	


	f = open(fasta)

	for chrfa in SeqIO.parse(f, "fasta"):
		Genome[chrfa.id] = chrfa.seq
		


	f.close()

In [7]:
def percent (c, total):
	try:
		return (100*float(c))/float(total)
	except ZeroDivisionError:
		return 0	

In [3]:
def PWM_to_dict(file):
	reader = csv.reader(file, delimiter = '\t')
	header = next(reader)
	header_dict = {}
	col = 0
	
	matrix = {}
	
	for name in header:
		header_dict[name] = col
		col += 1
	
	A_frec = []
	C_frec = []
	G_frec = []
	T_frec = []
	N_freq = []
	
	for row in reader:
		A = row[header_dict["A"]]
		C = row[header_dict["C"]]
		G = row[header_dict["G"]]
		T = row[header_dict["T"]]
		
		A_frec.append(float(A))
		C_frec.append(float(C))
		G_frec.append(float(G))
		T_frec.append(float(T))
		N_freq.append(0)
			
	matrix["A"] = A_frec
	matrix["C"] = C_frec
	matrix["G"] = G_frec
	matrix["T"] = T_frec
	matrix["N"] = N_freq
	
	return matrix

In [16]:
with open('../ME/hg19/gencode.v19.chr_patch_hapl_scaff.annotation.bed12') as bed, open('hg19.exon.intron.U2.upstream', "w") as out:

    reader = csv.reader(bed, delimiter='\t')
    
    out_csv = csv.writer(out, delimiter='\t')
    
    for row in reader:

        csv.field_size_limit(1000000000)
        
        transcript = row[3]

        qstarts = list(map (int, row[11].strip(",").split(",")))                      
        blocksizes = list(map(int, row[10].strip(",").split(",")))
        start = int(row[1])
        strand = row[5]
        bn = int(row[9])
        chr = row[0]
        
        intron_number = 0

        for q1, q2, b in zip(qstarts, qstarts[1:], blocksizes):
            istart = start + q1 + b
            iend = start + q2
            ilen = iend - istart
            intron = row[0] + ":" +  str(istart) + row[5] + str(iend)
            
            intron_number += 1
            
            
            out_csv.writerow([transcript, intron, chr, istart, iend, strand, intron_number, ilen])
            



# Are G4 enriched on short introns due to GC content?

In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna

Genome = {}
def Genomictabulator(fasta):
	


	f = open(fasta)

	for chrfa in SeqIO.parse(f, "fasta"):
		Genome[chrfa.id] = chrfa.seq
		


	f.close()

In [2]:
Genomictabulator("/Users/gp7/Genomes/hg19/GRCh37.p13.genome.fa")

In [3]:
def GC_calculator(seq):
    
    seq_str = str(seq).upper()
    GC_percent = percent(seq_str.count("G") + seq_str.count("C"), len(seq_str))
    return(GC_percent)

In [8]:

short_introns = set()
long_introns = set()

t = 500

with open('../ME/hg19/gencode.v19.chr_patch_hapl_scaff.annotation.bed12') as bed:

    reader = csv.reader(bed, delimiter='\t')
    
    for row in reader:

        csv.field_size_limit(1000000000)
        
        transcript = row[3]

        qstarts = list(map (int, row[11].strip(",").split(",")))                      
        blocksizes = list(map(int, row[10].strip(",").split(",")))
        start = int(row[1])
        strand = row[5]
        bn = int(row[9])
        chr = row[0]
        

        for q1, q2, b in zip(qstarts, qstarts[1:], blocksizes):
            istart = start + q1 + b
            iend = start + q2
            ilen = iend - istart
            intron = (row[0], row[5],  str(istart), str(iend))
            
            if strand == "+":
                GC = GC_calculator(Genome[chr][istart:iend])
            else:
                GC = GC_calculator(Genome[chr][istart:iend].reverse_complement())
                
                
            if "chr" in chr:
            
            
                if ilen < t:
                    short_introns.add((intron, GC))
                elif ilen > t:
                    long_introns.add((intron, GC))


In [9]:
import random 

with open('Intron_size/long_introns.same_GC.bed', "w") as out, open('Intron_size/short_introns.same_GC.bed', "w") as out2:

    for intron, GC in short_introns:

        long_intron_pool = list(filter(lambda x: x[1] > GC-0.5 and x[1] < GC+0.5 , long_introns))
        
        if len(long_intron_pool)==0:

            long_intron_pool = list(filter(lambda x: x[1] > GC-1 and x[1] < GC+1 , long_introns))

        if len(long_intron_pool)==0:

            long_intron_pool = list(filter(lambda x: x[1] > GC-2 and x[1] < GC+2 , long_introns))

        if len(long_intron_pool)==0:

            long_intron_pool = list(filter(lambda x: x[1] > GC-5 and x[1] < GC+5 , long_introns))

        if len(long_intron_pool)==0:

            long_intron_pool = list(filter(lambda x: x[1] > GC-10 and x[1] < GC+10 , long_introns))

        
        try:

            long_intron = random.choice(long_intron_pool)
            
            chrom, strand, start, end = long_intron[0]

            BED = [chrom, start, end, "_".join(long_intron[0]), str(long_intron[1]), strand ]

            out.write("\t".join(BED) + "\n")
        
            chrom_s, strand_s, start_s, end_s = intron
        
            BED_s = [chrom_s, str(start_s), str(end_s), "_".join(intron), str(GC), strand_s ]
        
            out2.write("\t".join(BED_s) + "\n")
        
            print(GC, str(long_intron[1]))
        
            long_introns.remove(long_intron)
            
        except IndexError:
            
            pass
            





64.19753086419753 63.78095887298341
65.21739130434783 65.25237746891003
53.587443946188344 53.16052751250569
62.264150943396224 62.5
58.167330677290835 58.201581027667984
44.60431654676259 44.557717411574565
34.959349593495936 35.21872387851769
57.6530612244898 57.89897904352499
67.01030927835052 66.66666666666667
55.55555555555556 55.76923076923077
60.83150984682713 60.679611650485434
52.40963855421687 52.11267605633803
58.22784810126582 58.168574401664934
31.9672131147541 31.53942428035044
64.23841059602648 64.70588235294117
32.51748251748252 32.96041308089501
57.475083056478404 57.847533632286996
50.588235294117645 50.84937712344281
29.722921914357684 29.968119022316685
54.36241610738255 54.509803921568626
53.40909090909091 53.36374002280502
54.973821989528794 54.85535140283345
54.41860465116279 53.99802566633761
66.9172932330827 66.66666666666667
64.98054474708171 64.8424543946932
69.41176470588235 69.72176759410802
69.87951807228916 70.36625971143174
44.10569105691057 43.770279039

47.674418604651166 47.57142857142857
57.01357466063349 57.34430082256169
45.45454545454545 45.82002423174358
46.45669291338583 46.394638615541226
42.285714285714285 42.158982786956145
72.32142857142857 72.1953578336557
52.35849056603774 52.22696766320927
31.524547803617573 31.51832460732984
59.036144578313255 59.12265996088293
58.59375 58.59232175502742
65.11627906976744 64.70203012442698
61.46341463414634 61.17290192113246
28.571428571428573 28.974069898534385
52.87356321839081 52.984094557128266
36.52173913043478 36.92870201096892
64.19098143236074 64.03033586132177
45.911949685534594 46.19215938303342
55.37190082644628 55.829858215179314
58.333333333333336 58.12236286919831
43.51851851851852 43.79496402877698
69.23076923076923 69.18480580452412
36.58536585365854 36.77573278800273
53.25443786982248 52.818532818532816
56.45161290322581 56.03146853146853
53.70741482965932 53.42177758445701
66.39676113360323 66.4869029275809
51.141552511415526 51.364764267990076
31.25 31.484641638225256

62.637362637362635 62.69967077185709
52.94117647058823 52.72978576364893
67.90450928381964 67.61565836298932
53.125 53.15243415802075
25.868725868725868 25.869894099848715
54.7008547008547 54.629629629629626
61.52019002375297 61.02594339622642
21.73913043478261 21.541501976284586
60.44444444444444 60.31927023945268
63.793103448275865 63.44086021505376
58.125 58.48142164781906
58.957654723127035 58.58156028368794
57.08955223880597 57.48987854251012
75.67567567567568 75.52581261950287
54.356846473029044 54.716577540106954
56.81818181818182 56.49171270718232
57.52212389380531 57.3213248111563
36.65768194070081 36.37699490346341
46.666666666666664 47.15871979098628
58.297872340425535 58.56950067476383
47.49034749034749 47.30473047304731
59.04059040590406 59.13555992141454
52.79503105590062 52.35920852359209
55.69620253164557 55.47378104875805
35.80246913580247 35.490660352538804
38.78504672897196 38.763408621493774
59.45205479452055 59.5213319458897
54.81171548117155 55.286343612334804
60.

54.0 54.45026178010471
52.147239263803684 51.8716577540107
36.75213675213675 37.16089978327479
37.89473684210526 37.753882915173236
46.97986577181208 47.009966777408636
58.73015873015873 59.092835947435354
45.78313253012048 46.08038201352964
54.3859649122807 54.779411764705884
59.67741935483871 59.972862957937586
61.965811965811966 61.48989898989899
28.45360824742268 28.60998650472335
22.82608695652174 23.23580034423408
38.888888888888886 39.00837672628481
58.778625954198475 58.69047619047619
45.69190600522193 45.250388683084125
29.003021148036254 29.479768786127167
53.44827586206897 53.91766268260292
57.79816513761468 58.18181818181818
62.5 62.13550600343053
44.26229508196721 43.91742442860654
31.11111111111111 31.3590692755156
71.42857142857143 71.62426614481409
56.75675675675676 56.79012345679013
60.451977401129945 59.96343692870201
30.0 29.840546697038725
36.22047244094488 36.1647499331372
41.935483870967744 41.976921402133684
32.0 32.32432432432432
55.95744680851064 55.50082101806

62.33766233766234 62.1342512908778
64.12213740458016 64.19322709163346
47.20496894409938 47.10578842315369
62.53968253968254 62.144702842377264
58.53658536585366 58.38616714697406
31.884057971014492 31.96523103065159
59.142857142857146 58.77034358047016
69.32270916334662 68.88556469708303
52.73224043715847 52.515337423312886
27.63157894736842 27.566539923954373
53.521126760563384 53.48199825588638
42.68292682926829 42.489626556016596
50.140845070422536 49.74379188017343
55.89430894308943 55.64808883194534
38.8235294117647 38.68977406768896
60.714285714285715 61.183098591549296
62.8755364806867 63.141171130276895
52.662721893491124 52.53164556962025
64.92537313432835 64.53900709219859
35.3356890459364 35.124378109452735
60.89108910891089 61.29032258064516
66.05839416058394 65.84158415841584
61.75298804780876 61.57354618015964
59.49367088607595 59.30138099106418
52.74390243902439 52.59391771019678
49.25373134328358 49.42212438084755
48.52607709750567 48.95136099955377
69.5364238410596 69

...

64.47368421052632 54.82280841993072
41.42857142857143 41.38666666666666
43.154761904761905 43.09963099630996
59.09090909090909 54.315027157513576
30.116959064327485 30.580865603644646
35.874439461883405 36.28323188179595
46.69926650366748 46.29753168779186
61.904761904761905 53.6847242557345
52.33160621761658 52.17391304347826
63.97849462365591 54.75047984644914
38.65546218487395 38.427947598253276
52.325581395348834 51.943005181347154
34.93975903614458 34.67261904761905
57.44047619047619 55.611601513240856
65.06024096385542 55.51330798479088
61.83206106870229 52.07631874298541
60.0 55.4089709762533
33.68421052631579 33.91153512575889
43.94904458598726 44.29530201342282
33.333333333333336 33.01985370950888
61.29032258064516 52.79187817258883
58.92116182572614 54.347497353697264
55.921052631578945 55.47054322876817
59.32203389830509 54.474708171206224
54.18410041841004 54.0888602704443
54.11764705882353 54.06293222683264
50.0 49.71006852925672
30.717488789237667 31.137724550898202
55.65

57.82312925170068 53.25468895866927
34.47204968944099 34.29014740108612
52.63157894736842 52.62718932443703
51.181102362204726 51.275820170109355
33.6283185840708 33.77091377091377
42.016806722689076 42.161933917449595
38.64734299516908 38.53464662173838
60.888888888888886 54.34173669467787
60.810810810810814 51.61957270847691
52.12765957446808 52.03252032520325
47.76119402985075 47.940257711831315
35.59322033898305 35.868811881188115
47.45762711864407 47.46028633065307
45.70637119113574 45.22630834512022
64.59627329192547 55.08637236084453
44.64751958224543 44.706778370144704
63.80597014925373 54.016620498614955
32.8 32.90608824254375
25.609756097560975 30.601383714741885
57.64705882352941 54.3777423214998
62.857142857142854 54.35516542876435
25.321888412017167 32.57211538461539
54.166666666666664 53.99087799315849
51.699029126213595 52.021857923497265
53.92156862745098 53.99182561307902
51.041666666666664 50.57152753723589
52.38095238095238 52.44618395303327
60.17316017316017 55.2741

44.79495268138801 45.24055059857418
53.48837209302326 53.30296127562642
47.17948717948718 46.709816612729234
48.45070422535211 48.77201686926321
41.41414141414141 41.84636118598383
63.43283582089552 53.96572915913528
24.899598393574298 34.62744152399325
65.05376344086021 55.20702634880803
40.40920716112532 40.6591767747067
64.58333333333333 54.661151765828826
52.608695652173914 53.06258322237017
35.19736842105263 35.10284587207664
35.064935064935064 35.04127899357882
62.5 53.63830076619667
43.66197183098591 43.287225386493084
23.333333333333332 32.08955223880597
64.21319796954315 54.30711610486891
64.83050847457628 55.42521994134897
48.529411764705884 48.27375496486404
33.05322128851541 33.148404993065185
61.74496644295302 54.621848739495796
51.14503816793893 50.95452967719542
50.0 49.9151679674245
31.218274111675125 31.1088295687885
53.333333333333336 53.82579403272377
63.577586206896555 54.4973544973545
56.35245901639344 55.384140697315644
59.611231101511876 54.76190476190476
56.5400

57.0945945945946 55.106167846309404
58.02469135802469 53.425712086220166
63.36206896551724 53.398058252427184
35.730858468677496 35.94144094747231
64.48202959830867 54.824561403508774
32.16374269005848 32.006472491909385
62.10526315789474 52.255685310387214
51.724137931034484 51.82809493264914
28.440366972477065 30.383480825958703
50.0 50.36101083032491
61.1764705882353 51.935336976320585
50.0 49.84678243105209
58.67768595041322 54.683293184072554
46.666666666666664 47.141119221411195
36.0 35.747126436781606
54.76190476190476 54.63035019455253
41.12149532710281 41.1751563896336
54.3026706231454 54.11710887465691
30.93525179856115 31.03448275862069
51.862464183381086 52.31732776617954
35.042735042735046 35.48943510133678
60.0 55.092592592592595
58.403361344537814 53.91907514450867
59.76744186046512 55.03615268202455
56.830601092896174 54.89913544668588
58.26086956521739 55.05424063116371


KeyboardInterrupt: 

In [ ]:
random.choice(list(filter(lambda x: x[1] > 50-1 and x[1] < 50+1 , long_intron)))

In [36]:
"_".join(("A", "B"))

'A_B'

In [15]:
number_list = set(range(-5, 5))
less_than_zero = list(filter(lambda x: x < 0, number_list))
print(less_than_zero)

[-1, -5, -4, -3, -2]
